In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

def bg_delete(img_rgb,bg_rgb,threshold=170):
    img=cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    bg=cv2.cvtColor(bg_rgb, cv2.COLOR_BGR2GRAY)

    diff=np.abs(img-bg)
    mask = cv2.inRange(diff, threshold, 255)
    kernel = np.ones((5,5),np.uint8)       
    mask_er = cv2.dilate(mask, kernel, iterations = 1)
    img_filt = cv2.bitwise_and(img_rgb, img_rgb, mask= mask_er)
    return img_filt

def bg_delete2(img,mean,std,threshold=200):
    mask=substract_color(img,mean,std)
    mask2 = cv2.inRange(mask, 200, 255)
    img_filt = cv2.bitwise_and(img, img, mask= mask2)
    return img_filt

# function substract
# input:     frame: frame in grayscale
#            mean: mean from background
#            std: std deviation from background
# Output:    mask of the detection
def substract_color(frame, mean, std,threshold=550):
    std += 1
    shape = np.shape(frame)
    difference = frame - mean
    difference_squared = np.multiply(difference, difference)
    mask = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            for k in range(shape[2]):
                if difference_squared[i,j,k]>threshold:#*std[i,j,k]:
                    mask[i,j,k] = 255
    mask2 = np.zeros((shape[0], shape[1]))
    for i in range(shape[0]):
        for j in range(shape[1]):
            if (mask[i,j,0]==255 or mask[i,j,1]==255 or mask[i,j,2]==255):
                mask2[i,j] = 255
                
    kernel = np.ones((5,5),np.uint8)       
    mask2 = cv2.erode(mask2, kernel, iterations = 2)

    return mask2

def border_penalty(x,y,w,h,img_w,img_h,thres):
    if y<thres:
        return True
    if x<thres:
        return True
    if x+w>img_w-thres:
        return True
    if y+h>img_h-thres:
        return True
    return False

In [3]:
def gen_bg_color(images, show=False):
    cam = cv2.VideoCapture(0)
    
    #Record first frame
    frame = images[0]
    shape = np.shape(frame)
    print(shape)
    aux = np.zeros((shape[0], shape[1], shape[2], 1))
    aux[:,:,:,0] = frame
    video = aux
    
    #Record the rest of the frames
    iteration=1
    for i in range(1,len(images)):
        frame = images[i]
        aux[:,:,:,0] = frame
        video = np.concatenate((video, aux), axis=3)

        if show:
            cv2.imshow('frame', frame)
        k = cv2.waitKey(30) & 0xff
        if k == 27 | iteration==frames:
            break
        iteration += 1
      
    #cam.release()
    cv2.destroyAllWindows()
    mean = np.mean(video, axis=3)
    std = np.std(video, axis=3)
    
    return mean, std

In [3]:
#!/usr/bin/env python

import sys
import cv2

# If image path and f/q is not passed as command
# line arguments, quit and display help message

# speed-up using multithreads
cv2.setUseOptimized(True);
cv2.setNumThreads(4);

bg= cv2.imread('db2/bg0.jpg',1)

# read image
k=0
while (k<2):
    k+=1
    if(k==1):
        print("--------------------Sin Fondo------------------")
    else:
        print("--------------------Con Fondo------------------")
    for j in range(0,15):
        im = cv2.imread('db2/img'+str(j)+'.jpg',1)
        if(k==1):
            im=bg_delete(im,bg)
        
        #cv2.imshow("Output", im)

        # key press
        #cv2.waitKey(0)

        #im = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # resize image
        newHeight = 200
        newWidth = int(im.shape[1]*200/im.shape[0])
        im = cv2.resize(im, (newWidth, newHeight))    

        # create Selective Search Segmentation Object using default parameters
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

        # set input image on which we will run segmentation
        ss.setBaseImage(im)

        # Switch to fast but low recall Selective Search method
        #ss.switchToSelectiveSearchFast()

        # Switch to high recall but slow Selective Search method
        ss.switchToSelectiveSearchQuality()
        # if argument is neither f nor q print help message

        # run selective search segmentation on input image
        rects = ss.process()
        print('Total Number of Region Proposals: {}'.format(len(rects)))

        # number of region proposals to show
        numShowRects = 10
        # increment to increase/decrease total number
        # of reason proposals to be shown
        increment = 10

        # create a copy of original image
        imOut = im.copy()

        # itereate over all the region proposals
        x_mean=0
        y_mean=0
        w_cum=0
        h_cum=0

        x_b=0
        y_b=0
        w_b=0
        h_b=0
        a_b=0

        for i, rect in enumerate(rects):
            # draw rectangle for region proposal till numShowRects
            if (i < numShowRects):
                x, y, w, h = rect
                cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)

                if (w<(2*newWidth/3) and h<(2*newHeight/3) and not border_penalty(x,y,w,h,newWidth,newHeight,5)):
                    x_mean+=x*w
                    y_mean+=y*h
                    w_cum+=w
                    h_cum+=h

                if(w*h>a_b and w<(2*newWidth/3) and h<(2*newHeight/3) and not border_penalty(x,y,w,h,newWidth,newHeight,5)):
                    x_b=x
                    y_b=y
                    w_b=w
                    h_b=h
                    a_b=w*h

            else:
                break
        x_mean=(x_mean/w_cum)
        y_mean=(y_mean/h_cum)

        cv2.rectangle(imOut, (int(x_mean-50), int(y_mean-50)), (int(x_mean-50)+100, int(y_mean-50)+100), (255, 255, 0), 1, cv2.LINE_AA)

        cv2.rectangle(imOut, (x_b, y_b), (x_b+w_b, y_b+h_b), (255, 0, 0), 1, cv2.LINE_AA)
        # show output
        if(k==1):
            cv2.imwrite("demos_imgs/img"+str(j*2)+".jpg", imOut)
        else:
            cv2.imwrite("demos_imgs/img"+str(j*2+1)+".jpg", imOut)
        
        
        #cv2.waitKey()

    #cv2.destroyAllWindows()

--------------------Sin Fondo------------------
Total Number of Region Proposals: 492
Total Number of Region Proposals: 1144
Total Number of Region Proposals: 1049
Total Number of Region Proposals: 957
Total Number of Region Proposals: 1565
Total Number of Region Proposals: 1486
Total Number of Region Proposals: 730
Total Number of Region Proposals: 682
Total Number of Region Proposals: 719
Total Number of Region Proposals: 1129
Total Number of Region Proposals: 931
Total Number of Region Proposals: 1013
Total Number of Region Proposals: 1424
Total Number of Region Proposals: 1453
Total Number of Region Proposals: 1240
--------------------Con Fondo------------------
Total Number of Region Proposals: 769
Total Number of Region Proposals: 1117
Total Number of Region Proposals: 1163
Total Number of Region Proposals: 1258
Total Number of Region Proposals: 1335
Total Number of Region Proposals: 1170
Total Number of Region Proposals: 671
Total Number of Region Proposals: 681
Total Number of 

In [4]:
#!/usr/bin/env python

import sys
import cv2

# If image path and f/q is not passed as command
# line arguments, quit and display help message

# speed-up using multithreads
cv2.setUseOptimized(True);
cv2.setNumThreads(4);

bg= cv2.imread('rasp_images/bg0.jpg',1)

# read image
k=0
while (k<2):
    k+=1
    if(k==1):
        print("--------------------Sin Fondo------------------")
    else:
        print("--------------------Con Fondo------------------")
    for j in range(0,30):
        im = cv2.imread('rasp_images/img'+str(j)+'.jpg',1)
        if(k==1):
            im=bg_delete(im,bg)
        
        #cv2.imshow("Output", im)

        # key press
        #cv2.waitKey(0)

        #im = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # resize image
        newHeight = 200
        newWidth = int(im.shape[1]*200/im.shape[0])
        im = cv2.resize(im, (newWidth, newHeight))    

        # create Selective Search Segmentation Object using default parameters
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

        # set input image on which we will run segmentation
        ss.setBaseImage(im)

        # Switch to fast but low recall Selective Search method
        #ss.switchToSelectiveSearchFast()

        # Switch to high recall but slow Selective Search method
        ss.switchToSelectiveSearchQuality()
        # if argument is neither f nor q print help message

        # run selective search segmentation on input image
        rects = ss.process()
        print('IT '+str(j)+': Total Number of Region Proposals: {}'.format(len(rects)))

        # number of region proposals to show
        numShowRects = 10
        # increment to increase/decrease total number
        # of reason proposals to be shown
        increment = 10

        # create a copy of original image
        imOut = im.copy()

        # itereate over all the region proposals
        x_mean=0
        y_mean=0
        w_cum=0
        h_cum=0

        x_b=0
        y_b=0
        w_b=0
        h_b=0
        a_b=0

        for i, rect in enumerate(rects):
            # draw rectangle for region proposal till numShowRects
            if (i < numShowRects):
                x, y, w, h = rect
                cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)

                if (w<(5*newWidth/6) and h<(5*newHeight/6) and not border_penalty(x,y,w,h,newWidth,newHeight,3)):
                    x_mean+=x*w
                    y_mean+=y*h
                    w_cum+=w
                    h_cum+=h

                if(w*h>a_b and not border_penalty(x,y,w,h,newWidth,newHeight,3)):
                    x_b=x
                    y_b=y
                    w_b=w
                    h_b=h
                    a_b=w*h
            else:
                break
        if (w_cum==0):
            w_cum=1
        if (h_cum==0):
            h_cum=1
        x_mean=(x_mean/w_cum)
        y_mean=(y_mean/h_cum)

        cv2.rectangle(imOut, (int(x_mean-50), int(y_mean-50)), (int(x_mean-50)+100, int(y_mean-50)+100), (255, 255, 0), 1, cv2.LINE_AA)

        cv2.rectangle(imOut, (x_b, y_b), (x_b+w_b, y_b+h_b), (255, 0, 0), 1, cv2.LINE_AA)
        # show output
        if(k==1):
            cv2.imwrite("demo_rasp_img/img"+str(j*2)+".jpg", imOut)
        else:
            cv2.imwrite("demo_rasp_img/img"+str(j*2+1)+".jpg", imOut)
        
        #cv2.waitKey()

    #cv2.destroyAllWindows()

--------------------Sin Fondo------------------
IT 0: Total Number of Region Proposals: 1591
IT 1: Total Number of Region Proposals: 1675
IT 2: Total Number of Region Proposals: 1260
IT 3: Total Number of Region Proposals: 1155
IT 4: Total Number of Region Proposals: 2176
IT 5: Total Number of Region Proposals: 1356
IT 6: Total Number of Region Proposals: 1800
IT 7: Total Number of Region Proposals: 1641
IT 8: Total Number of Region Proposals: 1960
IT 9: Total Number of Region Proposals: 1727
IT 10: Total Number of Region Proposals: 1340
IT 11: Total Number of Region Proposals: 1218
IT 12: Total Number of Region Proposals: 1461
IT 13: Total Number of Region Proposals: 2181
IT 14: Total Number of Region Proposals: 1379
IT 15: Total Number of Region Proposals: 903
IT 16: Total Number of Region Proposals: 1533
IT 17: Total Number of Region Proposals: 1025
IT 18: Total Number of Region Proposals: 1609
IT 19: Total Number of Region Proposals: 1641
IT 20: Total Number of Region Proposals: 82

In [4]:
#!/usr/bin/env python

import sys
import cv2

# If image path and f/q is not passed as command
# line arguments, quit and display help message

# speed-up using multithreads
cv2.setUseOptimized(True);
cv2.setNumThreads(4);

bg_images=[]
for i in range(0,4):
    bg= cv2.imread('rasp_images/bg'+str(i)+'.jpg',1)
    bg_images.append(bg)

mean, std = gen_bg_color(bg_images)

# read image
k=0
while (k<2):
    k+=1
    if(k==1):
        print("--------------------Sin Fondo------------------")
    else:
        print("--------------------Con Fondo------------------")
    for j in range(0,30):
        im = cv2.imread('rasp_images/img'+str(j)+'.jpg',1)
        if(k==1):
            im=bg_delete2(im,mean,std)
        
        #cv2.imshow("Output", im)

        # key press
        #cv2.waitKey(0)

        #im = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # resize image
        newHeight = 200
        newWidth = int(im.shape[1]*200/im.shape[0])
        im = cv2.resize(im, (newWidth, newHeight))    

        # create Selective Search Segmentation Object using default parameters
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

        # set input image on which we will run segmentation
        ss.setBaseImage(im)

        # Switch to fast but low recall Selective Search method
        #ss.switchToSelectiveSearchFast()

        # Switch to high recall but slow Selective Search method
        ss.switchToSelectiveSearchQuality()
        # if argument is neither f nor q print help message

        # run selective search segmentation on input image
        rects = ss.process()
        print('IT '+str(j)+': Total Number of Region Proposals: {}'.format(len(rects)))

        # number of region proposals to show
        numShowRects = 10
        # increment to increase/decrease total number
        # of reason proposals to be shown
        increment = 10

        # create a copy of original image
        imOut = im.copy()

        # itereate over all the region proposals
        x_mean=0
        y_mean=0
        w_cum=0
        h_cum=0

        x_b=0
        y_b=0
        w_b=0
        h_b=0
        a_b=0

        for i, rect in enumerate(rects):
            # draw rectangle for region proposal till numShowRects
            if (i < numShowRects):
                x, y, w, h = rect
                cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)

                if (w<(5*newWidth/6) and h<(5*newHeight/6) and not border_penalty(x,y,w,h,newWidth,newHeight,3)):
                    x_mean+=x*w
                    y_mean+=y*h
                    w_cum+=w
                    h_cum+=h

                if(w*h>a_b and not border_penalty(x,y,w,h,newWidth,newHeight,3)):
                    x_b=x
                    y_b=y
                    w_b=w
                    h_b=h
                    a_b=w*h
            else:
                break
        if (w_cum==0):
            w_cum=1
        if (h_cum==0):
            h_cum=1
        x_mean=(x_mean/w_cum)
        y_mean=(y_mean/h_cum)

        cv2.rectangle(imOut, (int(x_mean-50), int(y_mean-50)), (int(x_mean-50)+100, int(y_mean-50)+100), (255, 255, 0), 1, cv2.LINE_AA)

        cv2.rectangle(imOut, (x_b, y_b), (x_b+w_b, y_b+h_b), (255, 0, 0), 1, cv2.LINE_AA)
        # show output
        if(k==1):
            cv2.imwrite("demo_rasp_img2/img"+str(j*2)+".jpg", imOut)
        else:
            cv2.imwrite("demo_rasp_img2/img"+str(j*2+1)+".jpg", imOut)
        
        #cv2.waitKey()

    #cv2.destroyAllWindows()

(1080, 1920, 3)
--------------------Sin Fondo------------------
IT 0: Total Number of Region Proposals: 1036
IT 1: Total Number of Region Proposals: 1963
IT 2: Total Number of Region Proposals: 1894
IT 3: Total Number of Region Proposals: 2508
IT 4: Total Number of Region Proposals: 2729
IT 5: Total Number of Region Proposals: 2330
IT 6: Total Number of Region Proposals: 1894
IT 7: Total Number of Region Proposals: 3178
IT 8: Total Number of Region Proposals: 3169
IT 9: Total Number of Region Proposals: 2071
IT 10: Total Number of Region Proposals: 1837
IT 11: Total Number of Region Proposals: 1778
IT 12: Total Number of Region Proposals: 1773
IT 13: Total Number of Region Proposals: 2479
IT 14: Total Number of Region Proposals: 1288
IT 15: Total Number of Region Proposals: 1439
IT 16: Total Number of Region Proposals: 1777
IT 17: Total Number of Region Proposals: 1157
IT 18: Total Number of Region Proposals: 2020
IT 19: Total Number of Region Proposals: 2589
IT 20: Total Number of Reg